In [124]:
import tensorflow as tf
import numpy as np
import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split

In [125]:
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

--2024-04-10 19:13:02--  https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 

2606:4700:20::681a:221, 2606:4700:20::681a:321, 2606:4700:20::ac43:4695, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|2606:4700:20::681a:221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50264 (49K) [text/csv]
Saving to: ‘insurance.csv.6’

insurance.csv.6     100%[===================>]  49.09K  --.-KB/s    in 0.003s  

2024-04-10 19:13:02 (17.5 MB/s) - ‘insurance.csv.6’ saved [50264/50264]



,age,sex,bmi,children,smoker,region,expenses
1333,50,male,31.0,3,no,northwest,10600.55
1334,18,female,31.9,0,no,northeast,2205.98
1335,18,female,36.9,0,no,southeast,1629.83
1336,21,female,25.8,0,no,southwest,2007.95
1337,61,female,29.1,0,yes,northwest,29141.36


In [126]:
def change_sex (gender):
    if gender =='male':
        return 0
    else:
        return 1
    
dataset['sex'] = dataset['sex'].apply(lambda x: change_sex(x))

In [127]:
def change_sex (smoker):
    if smoker =='yes':
        return 0
    else:
        return 1
    
dataset['smoker'] = dataset['smoker'].apply(lambda x: change_sex(x))

In [128]:
dataset.tail()

,age,sex,bmi,children,smoker,region,expenses
1333,50,0,31.0,3,1,northwest,10600.55
1334,18,1,31.9,0,1,northeast,2205.98
1335,18,1,36.9,0,1,southeast,1629.83
1336,21,1,25.8,0,1,southwest,2007.95
1337,61,1,29.1,0,0,northwest,29141.36


In [129]:
dataset.describe()

,age,sex,bmi,children,smoker,expenses
count,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,0.494768,30.665471,1.094918,0.795217,13270.422414
std,14.049960,0.500160,6.098382,1.205493,0.403694,12110.011240
min,18.000000,0.000000,16.000000,0.000000,0.000000,1121.870000
25%,27.000000,0.000000,26.300000,0.000000,1.000000,4740.287500
50%,39.000000,0.000000,30.400000,1.000000,1.000000,9382.030000
75%,51.000000,1.000000,34.700000,2.000000,1.000000,16639.915000
max,64.000000,1.000000,53.100000,5.000000,1.000000,63770.430000


In [137]:
new_data = dataset[['age','sex','bmi','smoker','expenses']]
new_data.head()

from sklearn.preprocessing import StandardScaler

# Seleccionar las características para normalizar
features_to_normalize = ['age', 'bmi', 'expenses']  # Suponiendo que deseas normalizar estas características

# Normalizar solo las características seleccionadas
scaler = StandardScaler()
new_data[features_to_normalize] = scaler.fit_transform(new_data[features_to_normalize])

# Mostrar los primeros registros del DataFrame normalizado
print(new_data.head())


        age  sex       bmi  smoker  expenses
0 -1.438764    1 -0.453646       0  0.298583
1 -1.509965    0  0.514186       1 -0.953689
2 -0.797954    0  0.382954       1 -0.728675
3 -0.441948    0 -1.306650       1  0.719843
4 -0.513149    0 -0.289606       1 -0.776802


In [138]:
new_data.corr()

,age,sex,bmi,smoker,expenses
age,1.000000,0.020856,0.109341,0.025019,0.299008
sex,0.020856,1.000000,-0.046380,0.076185,-0.057292
bmi,0.109341,-0.046380,1.000000,-0.003968,0.198576
smoker,0.025019,0.076185,-0.003968,1.000000,-0.787251
expenses,0.299008,-0.057292,0.198576,-0.787251,1.000000


In [139]:
X = new_data[['age','sex','bmi','smoker']]
y = new_data['expenses']

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.20, random_state = 48)

In [140]:
# Construir el modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'), 
    tf.keras.layers.Dense(8, activation='relu'),   
    tf.keras.layers.Dense(1)  
])


# Compilar el modelo
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])  

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluar el modelo
loss, mae = model.evaluate(X_test, y_test)
print("Mean Squared Error:", loss)
print("Mean Absolute Error:", mae)

Epoch 1/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.8652 - mae: 0.6981 - val_loss: 0.4618 - val_mae: 0.4403
Epoch 2/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3303 - mae: 0.3382 - val_loss: 0.2008 - val_mae: 0.3133
Epoch 3/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1746 - mae: 0.2724 - val_loss: 0.1830 - val_mae: 0.2246
Epoch 4/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1544 - mae: 0.2179 - val_loss: 0.1692 - val_mae: 0.2578
Epoch 5/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1518 - mae: 0.2283 - val_loss: 0.1687 - val_mae: 0.2332
Epoch 6/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1561 - mae: 0.2293 - val_loss: 0.1721 - val_mae: 0.2085
Epoch 7/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1363 - mae: 0.2055 - val_loss: 0.1761 - val_mae: 0.2085
Epoch 8/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1310 - mae: 0.2077 - val_loss: 0.1646 - val_mae: 0.2172
Epoch 9/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1241 - mae: 

In [148]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae = model.evaluate(X_test, y_test, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")



9/9 - 0s - 3ms/step - loss: 0.1637 - mae: 0.2614
Testing set Mean Abs Error:  0.26 expenses
You passed the challenge. Great job!
